In [2]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.metrics import confusion_matrix, classification_report


In [3]:
report_dir = os.path.join("..", "report")
models_dir = os.path.join("..", "models")
os.makedirs(report_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

In [4]:
train_dir = "../data/raw/train"
class_names_all = sorted([
    d for d in os.listdir(train_dir)
    if os.path.isdir(os.path.join(train_dir, d))
])

In [5]:
def plot_and_save_history(history, title_prefix, out_prefix):
    if history is None or not hasattr(history, "history"):
        return
    hist = history.history
    # Accuracy
    if "accuracy" in hist and "val_accuracy" in hist:
        plt.figure(figsize=(8,4))
        plt.plot(hist["accuracy"], label="train")
        plt.plot(hist["val_accuracy"], label="val")
        plt.title(f"{title_prefix} — accuracy")
        plt.xlabel("Épocas")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(report_dir, f"{out_prefix}_accuracy.png"), dpi=150)
        plt.close()
    # Loss
    if "loss" in hist and "val_loss" in hist:
        plt.figure(figsize=(8,4))
        plt.plot(hist["loss"], label="train")
        plt.plot(hist["val_loss"], label="val")
        plt.title(f"{title_prefix} — loss")
        plt.xlabel("Épocas")
        plt.ylabel("Loss")
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(report_dir, f"{out_prefix}_loss.png"), dpi=150)
        plt.close()

def evaluate_and_report(model, dataset, class_names, prefix):
    # y_true
    y_true = np.concatenate([y.numpy() for _, y in dataset], axis=0)
    # y_pred
    y_pred = []
    for x, _ in dataset:
        probs = model.predict(x, verbose=0)
        y_pred.append(np.argmax(probs, axis=1))
    y_pred = np.concatenate(y_pred, axis=0)

    # accuracy
    acc = (y_true == y_pred).mean()

    # matriz de confusión
    cm = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicción")
    plt.ylabel("Real")
    plt.title(f"Matriz de confusión — {prefix} (acc={acc:.4f})")
    plt.tight_layout()
    plt.savefig(os.path.join(report_dir, f"{prefix}_confusion_matrix.png"), dpi=150)
    plt.close()

    # classification report
    report_str = classification_report(y_true, y_pred, target_names=class_names, digits=4)
    with open(os.path.join(report_dir, f"{prefix}_classification_report.txt"), "w", encoding="utf-8") as f:
        f.write(report_str)

    return acc, report_str

def plot_sample_predictions(model, dataset, class_names, prefix, max_show=16):
    # tomar 1 batch
    for images, labels in dataset.take(1):
        preds = np.argmax(model.predict(images, verbose=0), axis=1)
        n = min(max_show, images.shape[0])
        cols = 4
        rows = int(np.ceil(n/cols))
        plt.figure(figsize=(cols*3, rows*3))
        for i in range(n):
            plt.subplot(rows, cols, i+1)
            img = images[i].numpy()
            if img.shape[-1] == 1:
                plt.imshow(img.squeeze(), cmap="gray")
            else:
                plt.imshow(img.astype("float32"))
            true_c = class_names[int(labels[i])]
            pred_c = class_names[int(preds[i])]
            color = "green" if true_c == pred_c else "red"
            plt.title(f"T:{true_c}\nP:{pred_c}", color=color, fontsize=9)
            plt.axis("off")
        plt.tight_layout()
        plt.savefig(os.path.join(report_dir, f"{prefix}_sample_predictions.png"), dpi=150)
        plt.close()
        break

In [6]:
plot_and_save_history(globals().get("history", None), "CNN base", "cnn_base")
plot_and_save_history(globals().get("history_tl", None), "MobileNetV2 (head)", "mobilenet_head")
plot_and_save_history(globals().get("history_finetune", None), "MobileNetV2 (fine-tune)", "mobilenet_ft")


In [7]:
results = []
if "model" in globals() and "test_ds" in globals():
    acc_cnn, rep_cnn = evaluate_and_report(model, test_ds, class_names_all, "cnn_base")
    plot_sample_predictions(model, test_ds, class_names_all, "cnn_base")
    results.append(("CNN base", acc_cnn))
    print(f"[CNN base] Test accuracy: {acc_cnn:.4f}")
    print("[CNN base] classification_report guardado en ../report/cnn_base_classification_report.txt")


if "model_tl" in globals() and "test_ds_rgb" in globals():
    acc_tl, rep_tl = evaluate_and_report(model_tl, test_ds_rgb, class_names_all, "mobilenetv2_ft")
    plot_sample_predictions(model_tl, test_ds_rgb, class_names_all, "mobilenetv2_ft")
    results.append(("MobileNetV2 (FT)", acc_tl))
    print(f"[MobileNetV2 (FT)] Test accuracy: {acc_tl:.4f}")
    print("[MobileNetV2 (FT)] classification_report guardado en ../report/mobilenetv2_ft_classification_report.txt")


In [8]:
RETRAIN_SHORT = False
if RETRAIN_SHORT and "val_ds" in globals() and "train_ds" in globals() and "model" in globals():
    lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1
    )
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5, restore_best_weights=True
    )
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=10,  # corto
        callbacks=[lr_reduce, early_stop],
        verbose=1
    )

In [9]:
if "model_tl" in globals():
    converter = tf.lite.TFLiteConverter.from_keras_model(model_tl)
    tflite_model = converter.convert()
    tflite_path = os.path.join(models_dir, "modelo_mobilenetv2_finetuned.tflite")
    with open(tflite_path, "wb") as f:
        f.write(tflite_model)
    print(f"TFLite exportado en: {tflite_path}")


try:
    import tf2onnx
    if "model_tl" in globals():
        onnx_path = os.path.join(models_dir, "modelo_mobilenetv2_finetuned.onnx")
        spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
        model_proto, _ = tf2onnx.convert.from_keras(model_tl, input_signature=spec, output_path=onnx_path)
        print(f"ONNX exportado en: {onnx_path}")
except Exception as e:
    print("ONNX no exportado. Instala tf2onnx si lo necesitas: pip install tf2onnx")
    print(f"Detalle: {e}")

ONNX no exportado. Instala tf2onnx si lo necesitas: pip install tf2onnx
Detalle: No module named 'tf2onnx'


In [10]:
predict_py = r'''
import os
import sys
import numpy as np
import tensorflow as tf
from PIL import Image

# Uso: python predict.py <ruta_imagen> <ruta_modelo(.keras|.h5|.tflite)>
# Clases inferidas por orden alfabético de carpetas del train
TRAIN_DIR = os.path.join("..", "data", "raw", "train")
CLASS_NAMES = sorted([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))])

def load_image(path, target_size=(224, 224), color_mode="rgb"):
    img = Image.open(path).convert("RGB" if color_mode=="rgb" else "L")
    img = img.resize(target_size)
    arr = np.array(img).astype("float32") / 255.0
    if color_mode == "grayscale":
        if arr.ndim == 2:
            arr = np.expand_dims(arr, axis=-1)
    return np.expand_dims(arr, axis=0)

def predict_keras(model_path, image_path, color_mode="rgb", target_size=(224,224)):
    model = tf.keras.models.load_model(model_path)
    x = load_image(image_path, target_size=target_size, color_mode=color_mode)
    probs = model.predict(x, verbose=0)[0]
    idx = int(np.argmax(probs))
    return CLASS_NAMES[idx], float(probs[idx])

def predict_tflite(model_path, image_path, color_mode="rgb", target_size=(224,224)):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    x = load_image(image_path, target_size=target_size, color_mode=color_mode)
    x = x.astype(input_details[0]["dtype"])
    interpreter.set_tensor(input_details[0]["index"], x)
    interpreter.invoke()
    probs = interpreter.get_tensor(output_details[0]["index"])[0]
    idx = int(np.argmax(probs))
    return CLASS_NAMES[idx], float(probs[idx])

if __name__ == "__main__":
    if len(sys.argv) < 3:
        print("Uso: python predict.py <ruta_imagen> <ruta_modelo(.keras|.h5|.tflite)>")
        sys.exit(1)
    image_path = sys.argv[1]
    model_path = sys.argv[2]
    ext = os.path.splitext(model_path)[1].lower()
    if ext in [".keras", ".h5"]:
        # Asumimos MobileNetV2 fine-tuned (224,224,3) por defecto
        label, prob = predict_keras(model_path, image_path, color_mode="rgb", target_size=(224,224))
    elif ext == ".tflite":
        label, prob = predict_tflite(model_path, image_path, color_mode="rgb", target_size=(224,224))
    else:
        print("Formato no soportado:", ext)
        sys.exit(2)
    print(f"Predicción: {label} (confianza={prob:.4f})")
'''
# escribe el archivo
os.makedirs(os.path.join("..", "src"), exist_ok=True)
with open(os.path.join("..", "src", "predict.py"), "w", encoding="utf-8") as f:
    f.write(predict_py)

print("Archivo creado: ../src/predict.py")

Archivo creado: ../src/predict.py


In [11]:
if results:
    print("\nResumen test accuracy:")
    for name, acc in results:
        print(f"- {name}: {acc:.4f}")
print(f"Reportes guardados en: {report_dir}")
print(f"Modelos/exportaciones en: {models_dir}")

Reportes guardados en: ..\report
Modelos/exportaciones en: ..\models
